# Demo for paper "First Order Motion Model for Image Animation"
Cloned from the AliaksandrSiarohin repository, this notebook demonstrates the potential of their paper with few examples.

**Clone repository**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 216 (delta 0), reused 0 (delta 0), pack-reused 215
Receiving objects: 100% (216/216), 71.45 MiB | 38.31 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [ ]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [ ]:
data_dir = "/content/gdrive/My Drive/Colab Notebooks/DeepFake/data/"

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread(data_dir + '02.png')
driving_video = imageio.mimread(data_dir + '04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
def display_multiple(source, videos):
    fig = plt.figure(figsize=(4 + 4 * len(videos), 6))
    
    ims = []
    for i in range(len(videos[0])):
      cols = [source]
      for j in range(len(videos)):
        cols.append(videos[j][i])

      im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
      plt.axis('off')
      ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani    

**Create a model and load checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=data_dir + 'vox-cpk.pth.tar')

**Perfrorm image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:24<00:00,  8.57it/s]


**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:24<00:00,  8.48it/s]


## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [ ]:
source_image = imageio.imread(data_dir + 'angel.jpg')
source_image = resize(source_image, (256, 256))[..., :3]

In [ ]:
driving_video = imageio.mimread(data_dir + '04.mp4', memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:26<00:00,  8.01it/s]


In [ ]:
driving_video = imageio.mimread(data_dir + '00.mp4', memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 265/265 [00:33<00:00,  7.91it/s]


In [ ]:
!pip install pytube3 --upgrade

from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [ ]:
emma_video = YouTube('https://www.youtube.com/watch?v=gkjW9PZBRfk').streams.get_highest_resolution().download()
emma_output = "/content/emma.mp4"

ffmpeg_extract_subclip(emma_video, 120, 128, targetname="/content/emma_trim.mp4")
!ffmpeg -i "/content/emma_trim.mp4" -vf "crop=140:140:240:15" "/content/emma.mp4"


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/first-order-model/Emma Watson at the HeForShe Campaign 2014 - Official UN Video.mp4 -ss 120.00 -t 8.00 -vcodec copy -acodec copy /content/emma_trim.mp4
... command successful.
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enab

In [ ]:
driving_video = imageio.mimread(emma_output, memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 241/241 [00:30<00:00,  7.90it/s]


In [ ]:
greta_video = YouTube('https://www.youtube.com/watch?v=TMrtLsQbaok').streams.get_highest_resolution().download()
greta_output = "/content/greta.mp4"

ffmpeg_extract_subclip(greta_video, 46, 54, targetname="/content/greta_trim.mp4")
!ffmpeg -y -i "/content/greta_trim.mp4" -vf "crop=400:400:450:28" "/content/greta.mp4"


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/first-order-model/Greta Thunberg to world leaders How dare you You have stolen my dreams and my childhood.mp4 -ss 46.00 -t 8.00 -vcodec copy -acodec copy /content/greta_trim.mp4
... command successful.
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsv

In [ ]:
driving_video = imageio.mimread(greta_output, memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
driving_video = imageio.mimread(greta_output, memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 199/199 [00:25<00:00,  7.89it/s]


### Custom video

#### First crop it

In [ ]:
angel_video_1_output = "/content/angel_crop_1.mp4"
angel_video_2_output = "/content/angel_crop_2.mp4"
ffmpeg_extract_subclip("/content/gdrive/My Drive/Colab Notebooks/DeepFake/data/angel_1.mp4", 8, 15, targetname="/content/angel_trim_1.mp4")
!ffmpeg -y -i "/content/angel_trim_1.mp4" -vf "crop=420:420:750:75" "/content/angel_crop_1.mp4"


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/gdrive/My Drive/Colab Notebooks/DeepFake/data/angel_1.mp4 -ss 8.00 -t 7.00 -vcodec copy -acodec copy /content/angel_trim_1.mp4
... command successful.
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-lib

In [ ]:
source_image = imageio.imread(data_dir + 'bea_1.jpeg')
source_image = resize(source_image, (256, 256))[..., :3]

In [ ]:
driving_video = imageio.mimread(angel_video_1_output, memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
source_image_daenerys = imageio.imread(data_dir + 'got-05.png')
source_image_daenerys = resize(source_image_daenerys, (256, 256))[..., :3]

source_image_david = imageio.imread(data_dir + 'statue-02.png')
source_image_david  = resize(source_image_david, (256, 256))[..., :3]

source_image_shreek = imageio.imread(data_dir + 'cartoons-01.png')
source_image_shreek  = resize(source_image_shreek, (256, 256))[..., :3]

source_image_monalisa = imageio.imread(data_dir + '05.png')
source_image_monalisa = resize(source_image_monalisa, (256, 256))[..., :3]

source_image_it = imageio.imread(data_dir + 'doll-04.png')
source_image_it = resize(source_image_it, (256, 256))[..., :3]

In [ ]:
source_image_it = imageio.imread(data_dir + 'doll-04.png')
source_image_it = resize(source_image_it, (256, 256))[..., :3]

In [ ]:
driving_video = imageio.mimread(angel_video_1_output, memtest=False)
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions_daenerys = make_animation(source_image_daenerys, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)
predictions_david = make_animation(source_image_david, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)
predictions_shreek = make_animation(source_image_shreek, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)
predictions_monalisa = make_animation(source_image_monalisa, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)
predictions_monalisa = make_animation(source_image_monalisa, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)
predictions_it = make_animation(source_image_it, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

In [ ]:
HTML(display_multiple(source_image_daenerys, [driving_video, predictions_daenerys, predictions_david, predictions_monalisa, predictions_shreek, predictions_it]).to_html5_video())